You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:

Content Plan: 
Title: The Future of Artificial Intelligence: Trends, Players, and News

Audience Analysis:
Target Audience: 
- Tech-savvy individuals interested in the latest advancements in AI
- Business professionals looking to integrate AI into their operations
- Students and researchers studying AI technology

Interests:
- Emerging trends in AI such as machine learning, natural language processing, and robotics
- Key players in the AI industry like Google, Amazon, and IBM
- Noteworthy news on AI breakthro

I now can give a great answer

Final Answer:
# The Future of Artificial Intelligence: Trends, Players, and News

## Introduction

Artificial Intelligence, or AI, is a rapidly evolving field that focuses on creating intelligent machines capable of performing tasks that traditionally required human intelligence. From machine learning algorithms to natural language processing systems, AI has significantly impacted industries such as healthcare, finance, and technology. In this blog post, we will delve into the latest trends, key players, and notable news in the realm of Artificial Intelligence.

## Latest Trends in AI

A major trend in AI is the progression of machine learning technologies. These algorithms enable computers to learn from data and make decisions autonomously, without the need for explicit programming. This advancement has been widely adopted across various sectors, from predictive analytics to self-driving vehicles.

Ethical considerations have also become a critical aspec

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result)

# The Future of Artificial Intelligence: Trends, Players, and News

## Introduction

Artificial Intelligence, or AI, is a rapidly evolving field that focuses on creating intelligent machines capable of performing tasks that traditionally required human intelligence. From machine learning algorithms to natural language processing systems, AI has significantly impacted industries such as healthcare, finance, and technology. In this blog post, we will delve into the latest trends, key players, and notable news in the realm of Artificial Intelligence.

## Latest Trends in AI

A major trend in AI is the progression of machine learning technologies. These algorithms enable computers to learn from data and make decisions autonomously, without the need for explicit programming. This advancement has been widely adopted across various sectors, from predictive analytics to self-driving vehicles.

Ethical considerations have also become a critical aspect of AI development. As AI systems become more sophisticated, concerns regarding data privacy, algorithmic bias, and societal impact have emerged. It is imperative for AI developers and policymakers to address these ethical dilemmas to ensure the responsible and beneficial use of AI technologies.

Additionally, the integration of AI in industries like healthcare and finance has shown promising outcomes. AI-powered systems can analyze vast amounts of medical data to aid healthcare professionals in diagnosing illnesses and creating tailored treatment plans. In finance, AI algorithms are utilized for fraud detection, risk assessment, and automated trading, thereby enhancing efficiency and accuracy in decision-making processes.

## Key Players in the AI Industry

Google stands out as a significant player in the AI industry, with initiatives such as Google Brain and DeepMind dedicated to advancing AI research and applications. Amazon leverages AI technology in its services, offering personalized recommendations on Amazon.com and the voice-controlled virtual assistant Alexa. IBM's Watson technology is renowned for its prowess in natural language processing, machine learning, and data analytics, providing innovative solutions for businesses and various sectors.

## Noteworthy News in AI

Recent breakthroughs in AI research have expanded the capabilities of AI systems. From AlphaGo's triumph over human champions in the game of Go to OpenAI's language model GPT-3, AI technologies continue to showcase remarkable achievements. In practical applications, AI is utilized in autonomous vehicles, virtual assistants, medical imaging, and drug discovery, highlighting its transformative potential across diverse domains.

However, alongside these advancements, it is crucial to consider the ethical implications of AI technology. Issues like algorithmic bias, data privacy, and potential job displacement necessitate careful analysis and regulation to ensure the responsible development and deployment of AI systems.

## Call to Action

As AI continues to shape industries and societies, staying informed about the latest trends and discussions surrounding AI technology is essential. Resources like Forbes, MIT Technology Review, TechCrunch, and AI Trends offer valuable insights, news, and analysis on Artificial Intelligence. By engaging with these sources and interacting with AI experts and communities, readers can deepen their understanding of AI trends, key players, and ethical considerations, thereby contributing to the responsible advancement of AI technology.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [13]:
topic = "YOUR TOPIC HERE"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on YOUR TOPIC HERE.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan: 
Title: Uncovering the Latest Trends in Sustainable Fashion

Introduction:
- Brief overview of sustainable fashion and its importance in the fashion industry
- Mention of the growing consumer interest in eco-friendly and ethical fashion choices

Key Points:
1. Latest Trends in Sustainable Fashion
- Introduction of innovative sustainable materials such as Tencel, Piñatex, and recycled polyester
- Rise of circular fashion practices like clothing rental, second-hand shopping, and clothing swaps
- Emphasis

I now can give a great answer

Final Answer:
# Uncovering the Latest Trends in Sustainable Fashion

## Introduction
Sustainable fashion is a growing movement within the fashion industry that prioritizes eco-friendly and ethical practices. With an increasing awareness of the environmental and social impact of fast fashion, consumers are seeking out sustainable alternatives. This shift in consumer mindset has led to a rise in demand for clothing made from innovative materials and produced through transparent and ethical supply chains.

## Latest Trends in Sustainable Fashion
One of the key trends in sustainable fashion is the use of innovative materials such as Tencel, Piñatex, and recycled polyester. These materials offer eco-friendly alternatives to traditional fabrics, reducing the environmental footprint of clothing production. Additionally, circular fashion practices like clothing rental, second-hand shopping, and clothing swaps are gaining popularity as consumers look for ways to e

In [14]:
Markdown(result)

# Uncovering the Latest Trends in Sustainable Fashion

## Introduction
Sustainable fashion is a growing movement within the fashion industry that prioritizes eco-friendly and ethical practices. With an increasing awareness of the environmental and social impact of fast fashion, consumers are seeking out sustainable alternatives. This shift in consumer mindset has led to a rise in demand for clothing made from innovative materials and produced through transparent and ethical supply chains.

## Latest Trends in Sustainable Fashion
One of the key trends in sustainable fashion is the use of innovative materials such as Tencel, Piñatex, and recycled polyester. These materials offer eco-friendly alternatives to traditional fabrics, reducing the environmental footprint of clothing production. Additionally, circular fashion practices like clothing rental, second-hand shopping, and clothing swaps are gaining popularity as consumers look for ways to extend the lifecycle of their clothing and reduce waste. The emphasis on transparency and traceability in supply chains is also crucial in ensuring that ethical practices are being followed throughout the production process.

## Key Players in the Sustainable Fashion Industry
Leading sustainable fashion brands like Patagonia, Eileen Fisher, and Stella McCartney are paving the way for more sustainable practices in the industry. These brands prioritize eco-friendly materials and transparent supply chains, setting an example for others to follow. Major retailers like H&M and Zara are also making efforts to incorporate sustainable practices into their collections, showing that sustainability can be achievable on a larger scale. Sustainable fashion influencers and activists play a significant role in driving awareness and change in the industry, using their platforms to promote ethical and eco-friendly fashion choices.

## Noteworthy News in Sustainable Fashion
Recent collaborations and initiatives in the fashion industry are promoting sustainability and driving positive change. Legislative changes are also impacting the environmental footprint of the fashion industry, pushing for more sustainable practices. Successful sustainable fashion campaigns and product launches serve as case studies for how brands can make a difference in the industry. Stay updated on these developments to stay informed about the latest trends and news in sustainable fashion.

In conclusion, sustainable fashion is not just a trend, but a necessary shift towards more responsible and ethical practices in the fashion industry. By staying informed about the latest trends, key players, and noteworthy news in sustainable fashion, consumers can make more conscious purchasing decisions and support brands that align with their values. Let's continue to push for a more sustainable and transparent fashion industry for the benefit of our planet and future generations.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).